# Text classification : Comparison betwenn DistilBERT and CANINE

## Imports

In [1]:
import os
import numpy as np
import warnings

warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

In [ ]:
! pip install transformers
! pip install datasets

In [3]:
from datasets import load_dataset, load_metric
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

In [4]:
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import CanineTokenizer, CanineForSequenceClassification
from transformers import pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score

# IMDB dataset

## loading imdb datatset

We chose this dataset because it contains movie reviews. Therefore, it is common to have corruptions in the input due to natural typos. It is also considered informal text that often includes typos, spelling variations, transliterations or emoji. With this type of dataset, we expect better performance with CANINE, as it is designed to handle these problems as stated in the CANINE paper. 

In [ ]:
dataset = load_dataset("imdb")

In [6]:
def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(
        dataset
    ), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset) - 1)
        while pick in picks:
            pick = random.randint(0, len(dataset) - 1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [7]:
show_random_elements(dataset["train"])

In [8]:
sum(dataset["train"]["label"]) / len(dataset["train"]["label"])

0.5

In [9]:
sum(dataset["test"]["label"]) / len(dataset["train"]["label"])

0.5

## Metrics

In [ ]:
from datasets import load_metric

metric1 = load_metric("accuracy")
metric2 = load_metric("precision")
metric3 = load_metric("recall")


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    accuracy = metric1.compute(predictions=predictions, references=labels)["accuracy"]
    precision = metric2.compute(predictions=predictions, references=labels)["precision"]
    recall = metric3.compute(predictions=predictions, references=labels)["recall"]

    return {"accuracy": accuracy, "precision": precision, "recall": recall}

## distilbert-base-uncased-finetuned-sst-2-english for text classification on imdb 

This model is a fine-tune checkpoint of DistilBERT-base-uncased, fine-tuned on SST-2. This model reaches an accuracy of 91.3 on the dev set (for comparison, Bert bert-base-uncased version reaches an accuracy of 92.7).

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased-finetuned-sst-2-english", num_labels=2
)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    "distilbert-base-uncased-finetuned-sst-2-english"
)

In [ ]:
classifier = pipeline("text-classification", model=model, tokenizer=tokenizer, device=0)

In [ ]:
predictions = []
scores = []
for i in range(25000):
    single_pred = classifier(dataset["test"]["text"][i][:512])
    if single_pred[0]["label"] == "NEGATIVE":
        predictions.append(0)
    else:
        predictions.append(1)
    scores.append(single_pred[0]["score"])

In [ ]:
accuracy_score(dataset["test"]["label"], predictions)

0.82832

In [ ]:
precision_score(dataset["test"]["label"], predictions)

0.8483870967741935

In [ ]:
recall_score(dataset["test"]["label"], predictions)

0.79952

## canine-s-finetuned-sst2 for text classification on imdb 

canine-s-finetuned-sst2
This model is a fine-tuned version of google/canine-s on the glue dataset. It achieves the following results on the evaluation set: 
* Loss: 0.5259
* Accuracy: 0.8578

In [ ]:
model = CanineForSequenceClassification.from_pretrained(
    "celine98/canine-s-finetuned-sst2", num_labels=2
)
tokenizer = CanineTokenizer.from_pretrained("celine98/canine-s-finetuned-sst2")

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [ ]:
tokenized_dataset = dataset.map(preprocess_function, batched=True)

In [ ]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 50000
    })
})

In [ ]:
classifier = pipeline("text-classification", model=model, tokenizer=tokenizer, device=0)

In [ ]:
print(model)

CanineForSequenceClassification(
  (canine): CanineModel(
    (char_embeddings): CanineEmbeddings(
      (HashBucketCodepointEmbedder_0): Embedding(16384, 96)
      (HashBucketCodepointEmbedder_1): Embedding(16384, 96)
      (HashBucketCodepointEmbedder_2): Embedding(16384, 96)
      (HashBucketCodepointEmbedder_3): Embedding(16384, 96)
      (HashBucketCodepointEmbedder_4): Embedding(16384, 96)
      (HashBucketCodepointEmbedder_5): Embedding(16384, 96)
      (HashBucketCodepointEmbedder_6): Embedding(16384, 96)
      (HashBucketCodepointEmbedder_7): Embedding(16384, 96)
      (char_position_embeddings): Embedding(16384, 768)
      (token_type_embeddings): Embedding(16, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (initial_char_encoder): CanineEncoder(
      (layer): ModuleList(
        (0): CanineLayer(
          (attention): CanineAttention(
            (self): CanineSelfAttention(
            

In [ ]:
predictions = []
scores = []
for i in range(25000):
    single_pred = classifier(dataset["test"]["text"][i][:512])
    predictions.append(int(single_pred[0]["label"][-1]))
    scores.append(single_pred[0]["score"])

In [ ]:
accuracy_score(dataset["test"]["label"], predictions)

0.74556

In [ ]:
precision_score(dataset["test"]["label"], predictions)

0.846483801783497

In [ ]:
recall_score(dataset["test"]["label"], predictions)

0.59992

## canine-s for text classification using imdb

In [ ]:
tokenizer = CanineTokenizer.from_pretrained("google/canine-s")
model = CanineForSequenceClassification.from_pretrained("google/canine-s", num_labels=2)


def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)


# tokenize dataset
tokenized_dataset = dataset.map(preprocess_function, batched=True)

# creating a batch of examples
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding=True)

# Model was saved using *save_pretrained('./test/saved_model/')* (for example purposes, not runnable).
classifier = pipeline("text-classification", model=model, tokenizer=tokenizer, device=0)

# scores
predictions = []
scores = []
for i in range(25000):
    single_pred = classifier(dataset["test"]["text"][i][:512])
    predictions.append(int(single_pred[0]["label"][-1]))
    scores.append(single_pred[0]["score"])

In [ ]:
accuracy_score(dataset["test"]["label"], predictions)

0.50004

In [ ]:
precision_score(dataset["test"]["label"], predictions)

0.5000200056015685

In [ ]:
recall_score(dataset["test"]["label"], predictions)

0.99976

## finetuning all DistilBERT using imdb (text classification task)

In [ ]:
# Loading the DistilBERT tokenizer to process the text field

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [ ]:
# tokenizing text and truncating sequences to be no longer than DistilBERT’s maximum input length


def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [ ]:
tokenized_dataset = dataset.map(preprocess_function, batched=True)

In [ ]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 50000
    })
})

In [ ]:
# creating a batch of examples

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding=True)

In [ ]:
# distilbert-base-uncased model

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=2
)

In [ ]:
# Architecture

print(model)

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
       

**training distilbert-base-uncased model  using imdb dataset**

In [ ]:
training_args = TrainingArguments(
    evaluation_strategy="epoch",
    save_strategy="epoch",
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 25000
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 7815


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall
1,0.234300,0.215394,0.914880,0.959345,0.866480
2,0.144100,0.221827,0.928320,0.908017,0.953200
3,0.087900,0.323910,0.928560,0.945155,0.909920
4,0.045400,0.355530,0.928640,0.917355,0.942160
5,0.025000,0.385541,0.930760,0.927511,0.934560


The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 25000
  Batch size = 16
Saving model checkpoint to ./results/checkpoint-1563
Configuration saved in ./results/checkpoint-1563/config.json
Model weights saved in ./results/checkpoint-1563/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-1563/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-1563/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples =

TrainOutput(global_step=7815, training_loss=0.11321147417152683, metrics={'train_runtime': 4612.0295, 'train_samples_per_second': 27.103, 'train_steps_per_second': 1.694, 'total_flos': 1.64028791287344e+16, 'train_loss': 0.11321147417152683, 'epoch': 5.0})

In [ ]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 25000
  Batch size = 16


{'epoch': 5.0,
 'eval_loss': 0.19566546380519867,
 'eval_runtime': 444.1365,
 'eval_samples_per_second': 56.289,
 'eval_steps_per_second': 3.519}

**evaluation**

In [ ]:
# the dataset is balanced (accuracy can be used without problems):

print(sum(dataset["test"]["label"]))
print(len(dataset["test"]["label"]))

12500
25000


In [ ]:
! cp -r /content/Drive/MyDrive/NLPTraining/checkpoint-7815 ./checkpoint-7815

In [ ]:
# Model was saved using *save_pretrained('./test/saved_model/')* (for example purposes, not runnable).
model = AutoModelForSequenceClassification.from_pretrained(
    "/content/checkpoint-7815", num_labels=2
)
classifier = pipeline("text-classification", model=model, tokenizer=tokenizer, device=0)

In [ ]:
predictions = []
scores = []
for i in range(25000):
    single_pred = classifier(dataset["test"]["text"][i][:512])
    predictions.append(int(single_pred[0]["label"][-1]))
    scores.append(single_pred[0]["score"])

In [ ]:
accuracy_score(dataset["test"]["label"], predictions)

0.86784

In [ ]:
precision_score(dataset["test"]["label"], predictions)

0.8606840288672732

In [ ]:
recall_score(dataset["test"]["label"], predictions)

0.87776

## finetuning most CANINE using imbd (text classification task)

In [ ]:
list_para = [
    "classifier.weight",
    "classifier.bias",
    "canine.projection.conv.weight",
    "canine.projection.conv.bias",
    "canine.projection.LayerNorm.weight",
    "canine.projection.LayerNorm.bias",
    "canine.final_char_encoder.layer.0.attention.self.query.weight",
    "canine.final_char_encoder.layer.0.attention.self.query.bias",
    "canine.final_char_encoder.layer.0.attention.self.key.weight",
    "canine.final_char_encoder.layer.0.attention.self.key.bias",
    "canine.final_char_encoder.layer.0.attention.self.value.weight",
    "canine.final_char_encoder.layer.0.attention.self.value.bias",
    "canine.final_char_encoder.layer.0.attention.output.dense.weight",
    "canine.final_char_encoder.layer.0.attention.output.dense.bias",
    "canine.final_char_encoder.layer.0.attention.output.LayerNorm.weight",
    "canine.final_char_encoder.layer.0.attention.output.LayerNorm.bias",
    "canine.final_char_encoder.layer.0.intermediate.dense.weight",
    "canine.final_char_encoder.layer.0.intermediate.dense.bias",
    "canine.final_char_encoder.layer.0.output.dense.weight",
    "canine.final_char_encoder.layer.0.output.dense.bias",
    "canine.final_char_encoder.layer.0.output.LayerNorm.weight",
    "canine.final_char_encoder.layer.0.output.LayerNorm.bias",
    "canine.pooler.dense.weight",
    "canine.pooler.dense.bias",
]

In [ ]:
tokenizer = CanineTokenizer.from_pretrained("google/canine-s")
model = CanineForSequenceClassification.from_pretrained("google/canine-s", num_labels=2)

for name, param in model.named_parameters():
    if name not in list_para:
        print(name)
        param.requires_grad = False


def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)


tokenized_dataset = dataset.map(preprocess_function, batched=True)

# creating a batch of examples

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding=True)

In [ ]:
training_args = TrainingArguments(
    evaluation_strategy="epoch",
    save_strategy="epoch",
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `CanineForSequenceClassification.forward` and have been ignored: text. If text are not expected by `CanineForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 25000
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 7815


Epoch,Training Loss,Validation Loss
1,0.689400,0.684168
2,0.683300,0.679061
3,0.679000,0.674761
4,0.678700,0.672913
5,0.676400,0.672415


The following columns in the evaluation set  don't have a corresponding argument in `CanineForSequenceClassification.forward` and have been ignored: text. If text are not expected by `CanineForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 25000
  Batch size = 16
Saving model checkpoint to ./results/checkpoint-1563
Configuration saved in ./results/checkpoint-1563/config.json
Model weights saved in ./results/checkpoint-1563/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-1563/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-1563/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `CanineForSequenceClassification.forward` and have been ignored: text. If text are not expected by `CanineForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 25000
  Batch s

TrainOutput(global_step=7815, training_loss=0.6834106289112484, metrics={'train_runtime': 9685.1513, 'train_samples_per_second': 12.906, 'train_steps_per_second': 0.807, 'total_flos': 1.6315300305358272e+17, 'train_loss': 0.6834106289112484, 'epoch': 5.0})

In [ ]:
# Model was saved using *save_pretrained('./test/saved_model/')* (for example purposes, not runnable).
classifier = pipeline("text-classification", model=model, tokenizer=tokenizer, device=0)

# scores
predictions = []
scores = []
for i in range(25000):
    single_pred = classifier(dataset["test"]["text"][i][:512])
    predictions.append(int(single_pred[0]["label"][-1]))
    scores.append(single_pred[0]["score"])

In [ ]:
accuracy_score(dataset["test"]["label"], predictions)

0.57688

In [ ]:
precision_score(dataset["test"]["label"], predictions)

0.560100062539087

In [ ]:
recall_score(dataset["test"]["label"], predictions)

0.71648

## finetuning distilbert (classifier only) using imbd (text classification task) 

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")


def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)


tokenized_dataset = dataset.map(preprocess_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding=True)
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=2
)

In [ ]:
print(model)

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
       

In [ ]:
for name, param in model.named_parameters():
    if name not in [
        "pre_classifier.weight",
        "pre_classifier.bias",
        "classifier.weight",
        "classifier.bias",
    ]:
        print(name)
        param.requires_grad = False

In [ ]:
training_args = TrainingArguments(
    evaluation_strategy="epoch",
    save_strategy="epoch",
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 25000
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 7815


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall
1,0.452700,0.411329,0.829280,0.835343,0.820240
2,0.387500,0.368000,0.841400,0.830379,0.858080
3,0.375900,0.352435,0.847080,0.847664,0.846240
4,0.368600,0.348280,0.849680,0.845916,0.855120
5,0.359800,0.346832,0.850480,0.846488,0.856240


The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 25000
  Batch size = 16
Saving model checkpoint to ./results/checkpoint-1563
Configuration saved in ./results/checkpoint-1563/config.json
Model weights saved in ./results/checkpoint-1563/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-1563/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-1563/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples =

TrainOutput(global_step=7815, training_loss=0.4057229557635307, metrics={'train_runtime': 2492.2056, 'train_samples_per_second': 50.156, 'train_steps_per_second': 3.136, 'total_flos': 1.64028791287344e+16, 'train_loss': 0.4057229557635307, 'epoch': 5.0})

In [ ]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 25000
  Batch size = 16


{'epoch': 5.0,
 'eval_accuracy': 0.85048,
 'eval_loss': 0.3468317687511444,
 'eval_precision': 0.8464884530211958,
 'eval_recall': 0.85624,
 'eval_runtime': 240.1053,
 'eval_samples_per_second': 104.121,
 'eval_steps_per_second': 6.51}

## finetuning CANINE (classifier only) using imbd (text classification task)

In [ ]:
tokenizer = CanineTokenizer.from_pretrained("google/canine-s")
model = CanineForSequenceClassification.from_pretrained("google/canine-s", num_labels=2)

In [ ]:
# Architecture

print(model)

CanineForSequenceClassification(
  (canine): CanineModel(
    (char_embeddings): CanineEmbeddings(
      (HashBucketCodepointEmbedder_0): Embedding(16384, 96)
      (HashBucketCodepointEmbedder_1): Embedding(16384, 96)
      (HashBucketCodepointEmbedder_2): Embedding(16384, 96)
      (HashBucketCodepointEmbedder_3): Embedding(16384, 96)
      (HashBucketCodepointEmbedder_4): Embedding(16384, 96)
      (HashBucketCodepointEmbedder_5): Embedding(16384, 96)
      (HashBucketCodepointEmbedder_6): Embedding(16384, 96)
      (HashBucketCodepointEmbedder_7): Embedding(16384, 96)
      (char_position_embeddings): Embedding(16384, 768)
      (token_type_embeddings): Embedding(16, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (initial_char_encoder): CanineEncoder(
      (layer): ModuleList(
        (0): CanineLayer(
          (attention): CanineAttention(
            (self): CanineSelfAttention(
            

In [ ]:
for name, param in model.named_parameters():
    if name not in ["classifier.weight", "classifier.bias"]:
        print(name)
        param.requires_grad = False

canine.char_embeddings.HashBucketCodepointEmbedder_0.weight
canine.char_embeddings.HashBucketCodepointEmbedder_1.weight
canine.char_embeddings.HashBucketCodepointEmbedder_2.weight
canine.char_embeddings.HashBucketCodepointEmbedder_3.weight
canine.char_embeddings.HashBucketCodepointEmbedder_4.weight
canine.char_embeddings.HashBucketCodepointEmbedder_5.weight
canine.char_embeddings.HashBucketCodepointEmbedder_6.weight
canine.char_embeddings.HashBucketCodepointEmbedder_7.weight
canine.char_embeddings.char_position_embeddings.weight
canine.char_embeddings.token_type_embeddings.weight
canine.char_embeddings.LayerNorm.weight
canine.char_embeddings.LayerNorm.bias
canine.initial_char_encoder.layer.0.attention.self.query.weight
canine.initial_char_encoder.layer.0.attention.self.query.bias
canine.initial_char_encoder.layer.0.attention.self.key.weight
canine.initial_char_encoder.layer.0.attention.self.key.bias
canine.initial_char_encoder.layer.0.attention.self.value.weight
canine.initial_char_enc

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [ ]:
tokenized_dataset = dataset.map(preprocess_function, batched=True)

In [ ]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 50000
    })
})

In [ ]:
# creating a batch of examples

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding=True)

In [ ]:
training_args = TrainingArguments(
    evaluation_strategy="epoch",
    save_strategy="epoch",
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `CanineForSequenceClassification.forward` and have been ignored: text. If text are not expected by `CanineForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 25000
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 7815


Epoch,Training Loss,Validation Loss
1,0.696600,0.694117
2,0.696500,0.693941
3,0.695300,0.692565
4,0.693500,0.692273
5,0.694600,0.692236


The following columns in the evaluation set  don't have a corresponding argument in `CanineForSequenceClassification.forward` and have been ignored: text. If text are not expected by `CanineForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 25000
  Batch size = 16
Saving model checkpoint to ./results/checkpoint-1563
Configuration saved in ./results/checkpoint-1563/config.json
Model weights saved in ./results/checkpoint-1563/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-1563/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-1563/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `CanineForSequenceClassification.forward` and have been ignored: text. If text are not expected by `CanineForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 25000
  Batch s

TrainOutput(global_step=7815, training_loss=0.6959521703055023, metrics={'train_runtime': 9700.3192, 'train_samples_per_second': 12.886, 'train_steps_per_second': 0.806, 'total_flos': 1.6315300305358272e+17, 'train_loss': 0.6959521703055023, 'epoch': 5.0})

In [ ]:
# Model was saved using *save_pretrained('./test/saved_model/')* (for example purposes, not runnable).
classifier = pipeline("text-classification", model=model, tokenizer=tokenizer, device=0)

In [ ]:
predictions = []
scores = []
for i in range(25000):
    single_pred = classifier(dataset["test"]["text"][i][:512])
    predictions.append(int(single_pred[0]["label"][-1]))
    scores.append(single_pred[0]["score"])

In [ ]:
accuracy_score(dataset["test"]["label"], predictions)

0.52728

In [ ]:
precision_score(dataset["test"]["label"], predictions)

0.547812675266405

In [ ]:
recall_score(dataset["test"]["label"], predictions)

0.31256

## finetuning canine-s-finetuned-sst2 using imbd (text classification task)

In [ ]:
model = CanineForSequenceClassification.from_pretrained(
    "celine98/canine-s-finetuned-sst2", num_labels=2
)
tokenizer = CanineTokenizer.from_pretrained("celine98/canine-s-finetuned-sst2")

In [12]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [ ]:
tokenized_dataset = dataset.map(preprocess_function, batched=True)

In [19]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding=True)

In [20]:
for name, param in model.named_parameters():
    if name not in ["classifier.weight", "classifier.bias"]:
        print(name)
        param.requires_grad = False

canine.char_embeddings.HashBucketCodepointEmbedder_0.weight
canine.char_embeddings.HashBucketCodepointEmbedder_1.weight
canine.char_embeddings.HashBucketCodepointEmbedder_2.weight
canine.char_embeddings.HashBucketCodepointEmbedder_3.weight
canine.char_embeddings.HashBucketCodepointEmbedder_4.weight
canine.char_embeddings.HashBucketCodepointEmbedder_5.weight
canine.char_embeddings.HashBucketCodepointEmbedder_6.weight
canine.char_embeddings.HashBucketCodepointEmbedder_7.weight
canine.char_embeddings.char_position_embeddings.weight
canine.char_embeddings.token_type_embeddings.weight
canine.char_embeddings.LayerNorm.weight
canine.char_embeddings.LayerNorm.bias
canine.initial_char_encoder.layer.0.attention.self.query.weight
canine.initial_char_encoder.layer.0.attention.self.query.bias
canine.initial_char_encoder.layer.0.attention.self.key.weight
canine.initial_char_encoder.layer.0.attention.self.key.bias
canine.initial_char_encoder.layer.0.attention.self.value.weight
canine.initial_char_enc

In [16]:
training_args = TrainingArguments(
    evaluation_strategy="epoch",
    save_strategy="epoch",
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
)

In [17]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()